# Analisis de Egresos Hospitalarios

In [1]:
import importlib

import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

import lectura_archivos as fr
import calculo_metricas as aux

from functools import reduce

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

importlib.reload(fr)
importlib.reload(aux)


<module 'calculo_metricas' from 'c:\\Users\\ppizarro\\Desktop\\Javier Rojas\\Javier\\trabajo\\SSMO\\EPH\\modulo_gestion\\1.analisis_problemas_de_salud\\egresos-hospitalarios\\calculo_metricas.py'>

In [2]:
HOSPITAL_A_ANALIZAR = 112103  # Este es el Torax


## Lectura de archivos

- Se leen todos los archivos que estan en input/utf-8. Los archivos estan separados por ;
- Se dejan solamente los egresos de los diagnosticos que **SI** tenga el Tórax
- Se cambia la codificacion de INTERV_Q de 2 (No, 1 es SI) a 0.
- Se cambia la codificacion de CONDICION_EGRESO (orig 1 = Vivo, 2 = Fallecido) de 1 -> 0 y 2 -> 1
- Se cambia la codificacion de GLOSA_REGION_RESIDENCIA {
                        "Del Libertador B. O'Higgins": "del Libertador General Bernardo O'Higgins",
                        "De Aisén del Gral. C. Ibáñez del Campo": "Aysén del General Carlos Ibáñez del Campo",
                    }

In [3]:
df = fr.leer_archivos(HOSPITAL_A_ANALIZAR)

# Analisis de Egresos Hospitalarios

Se calculan las siguientes metricas:

- Número de Egresos (n_egresos)
- Días de Estada promedio (dias_estada_promedio)
- Cantidad de Intervenciones Quirúrgicas (n_int_q)
- Cantidad de Muertes (n_muertos)

Todas estas metricas estan calculadas a nivel de Año, Establecimiento de Salud y Diagnóstico. Por
ejemplo:

***"En el año 2019, el Hospital San José (código XXXXXXX) tuvo XX egresos, con XX dias estada_promedio,
con XX intervenciones_quirúrgicas y XX muertos para el diagnóstico A24.4."***

Este análisis se realizó entre 2001 y 2020 para todos los hospitales/clínicas de Chile.

In [4]:
AGRUPACION = [
    "ANO_EGRESO",
    "ESTABLECIMIENTO_SALUD",
    "GLOSA_ESTABLECIMIENTO_SALUD",
    "DIAG1",
]

metricas = aux.obtener_metricas_egresos(df, AGRUPACION).collect(streaming=True)


# Analisis de Estratos

In [5]:
variables_a_rankear = ["n_egresos"]
subgrupo_del_ranking = ["ANO_EGRESO", "DIAG1"]
unir_tablas_por = AGRUPACION

In [6]:
dict_estratos = aux.obtener_diccionario_estratos(df, HOSPITAL_A_ANALIZAR)

In [7]:
dict_resultados_estratos = aux.obtener_resumen_por_estratos(
    metricas, dict_estratos, variables_a_rankear, subgrupo_del_ranking
)

resultado = reduce(aux.left_join_consecutivo, dict_resultados_estratos.values())

# Analisis Sociodemográfico

In [8]:
desglose_sociodemografico = [
    "ANO_EGRESO",
    "REGION_RESIDENCIA",
    "GLOSA_REGION_RESIDENCIA",
    "region_pais",
    "COMUNA_RESIDENCIA",
    "GLOSA_COMUNA_RESIDENCIA",
    "comuna_region_pais",
    "DIAG1",
]

variables_a_analizar = [
    "DIAG1",
    "SEXO",
    "PUEBLO_ORIGINARIO",
    "GLOSA_PAIS_ORIGEN",
    "PREVISION",
    "BENEFICIARIO",
]

In [21]:
with pl.StringCache():
    df_socio = (
        pl.read_csv("input/utf-8/*.csv", separator=";", dtypes=fr.DICT_VARIABLES)
        .lazy()
        .filter(pl.col("ESTABLECIMIENTO_SALUD") == 112103)
        .with_columns(
            pl.col("GLOSA_REGION_RESIDENCIA").map_dict(
                {
                    "Del Libertador B. O'Higgins": "del Libertador General Bernardo O'Higgins",
                    "De Aisén del Gral. C. Ibáñez del Campo": "Aysén del General Carlos Ibáñez del Campo",
                },
                default=pl.col("GLOSA_REGION_RESIDENCIA"),
            )
        )
        .with_columns(
            (("Región " + pl.col("GLOSA_REGION_RESIDENCIA") + ", Chile").alias("region_pais"))
        )
        .with_columns(
            (pl.col("GLOSA_COMUNA_RESIDENCIA") + ", " + pl.col("region_pais")).alias(
                "comuna_region_pais"
            )
        )
        .with_columns((pl.col("EDAD_A_OS")))
        .collect()
    )

    # dict_resultados = {}

    # for variable in variables_a_analizar:
    #     agrupacion_total = desglose_sociodemografico.copy()
    #     if variable != "DIAG1":
    #         agrupacion_total += [variable]

    #     resultado = (
    #         df_socio.groupby(agrupacion_total)
    #         .agg(pl.col(variable).count().alias("conteo"))
    #         .sort(["ANO_EGRESO", "DIAG1", "conteo"], descending=True)
    #         .with_columns(
    #             pl.concat_str(pl.col(desglose_sociodemografico), separator="-").alias("llave_id")
    #         )
    #     ).collect()

    #     dict_resultados[variable] = resultado

In [10]:
import xlsxwriter

with xlsxwriter.Workbook("output/distribucion_sociodemografica/distribucion_polars.xlsx") as f:
    for var, df in dict_resultados.items():
        df.write_excel(workbook=f, worksheet=var)


In [23]:
df_socio.join(
    pl.cut(df_socio.get_column("EDAD_A_OS"), bins=range(0, 101, 10)).with_columns(
        pl.col("EDAD_A_OS").cast(pl.Int64())), on='EDAD_A_OS'
    )


C:\Users\ppizarro\AppData\Local\Temp\ipykernel_4648\2729992260.py:2: DeprecationWarning: `pl.cut(series)` has been deprecated; use `series.cut()`
  pl.cut(df_socio.get_column("EDAD_A_OS"), bins=range(0, 101, 10)).with_columns(


ComputeError: datatypes of join keys don't match

In [33]:
df_socio.with_columns(
    (df_socio.get_column("EDAD_A_OS")).cut(bins=range(0, 101, 10), maintain_order=True).select(
      pl.col('category').alias('EDAD_CATEGORIA')
    )
)

ID_PACIENTE,ESTABLECIMIENTO_SALUD,GLOSA_ESTABLECIMIENTO_SALUD,PERTENENCIA_ESTABLECIMIENTO_SALUD,SEREMI,SERVICIO_DE_SALUD,SEXO,FECHA_NACIMIENTO,EDAD_CANT,TIPO_EDAD,EDAD_A_OS,PUEBLO_ORIGINARIO,PAIS_ORIGEN,GLOSA_PAIS_ORIGEN,COMUNA_RESIDENCIA,GLOSA_COMUNA_RESIDENCIA,REGION_RESIDENCIA,GLOSA_REGION_RESIDENCIA,PREVISION,BENEFICIARIO,MODALIDAD,PROCEDENCIA,ANO_EGRESO,FECHA_EGRESO,AREA_FUNCIONAL_EGRESO,DIAS_ESTADA,CONDICION_EGRESO,DIAG1,GLOSA_DIAG1,DIAG2,GLOSA_DIAG2,INTERV_Q,CODIGO_INTERV_Q_PPAL,GLOSA_INTERV_Q_PPAL,PROCED,CODIGO_PROCED_PPAL,GLOSA_PROCED_PPAL,region_pais,comuna_region_pais,EDAD_CATEGORIA
str,i32,cat,cat,i8,i8,i8,str,i8,i8,i8,i8,i16,str,i64,cat,cat,cat,i8,cat,i8,i8,i16,date,i16,i16,i8,str,str,str,str,i8,i32,str,cat,cat,cat,str,str,cat
"""NA""",112103,"""Instituto Naci…","""Pertenecientes…",null,12,1,null,null,null,51,null,null,null,13123,"""Providencia""","""13""","""Metropolitana …",99,null,null,null,2001,2001-05-25,111,2,2,"""A162""","""TUBERCULOSIS D…",null,null,2,null,null,null,null,null,"""Región Metropo…","""Providencia, R…","""(50.0, 60.0]"""
"""NA""",112103,"""Instituto Naci…","""Pertenecientes…",null,12,2,null,null,null,95,null,null,null,13123,"""Providencia""","""13""","""Metropolitana …",99,null,null,null,2001,2001-07-16,111,3,2,"""J969""","""INSUFICIENCIA …",null,null,2,null,null,null,null,null,"""Región Metropo…","""Providencia, R…","""(90.0, 100.0]"""
"""NA""",112103,"""Instituto Naci…","""Pertenecientes…",null,12,1,null,null,null,65,null,null,null,13123,"""Providencia""","""13""","""Metropolitana …",1,"""A""",null,null,2001,2001-04-19,270,2,2,"""J189""","""NEUMONIA, NO E…",null,null,2,null,null,null,null,null,"""Región Metropo…","""Providencia, R…","""(60.0, 70.0]"""
"""NA""",112103,"""Instituto Naci…","""Pertenecientes…",null,12,1,null,null,null,48,null,null,null,13123,"""Providencia""","""13""","""Metropolitana …",1,"""A""",null,null,2001,2001-09-19,270,45,2,"""J189""","""NEUMONIA, NO E…",null,null,1,null,null,null,null,null,"""Región Metropo…","""Providencia, R…","""(40.0, 50.0]"""
"""NA""",112103,"""Instituto Naci…","""Pertenecientes…",null,12,2,null,null,null,65,null,null,null,13123,"""Providencia""","""13""","""Metropolitana …",1,"""A""",null,null,2001,2001-04-13,111,4,2,"""J189""","""NEUMONIA, NO E…",null,null,2,null,null,null,null,null,"""Región Metropo…","""Providencia, R…","""(60.0, 70.0]"""
"""NA""",112103,"""Instituto Naci…","""Pertenecientes…",null,12,2,null,null,null,49,null,null,null,13124,"""Pudahuel""","""13""","""Metropolitana …",1,"""B""",null,null,2001,2001-05-19,111,17,2,"""I891""","""LINFANGITIS""",null,null,2,null,null,null,null,null,"""Región Metropo…","""Pudahuel, Regi…","""(40.0, 50.0]"""
"""NA""",112103,"""Instituto Naci…","""Pertenecientes…",null,12,1,null,null,null,23,null,null,null,13101,"""Santiago""","""13""","""Metropolitana …",1,"""A""",null,null,2001,2001-04-25,111,6,2,"""B59X""","""NEUMOCISTOSIS""",null,null,2,null,null,null,null,null,"""Región Metropo…","""Santiago, Regi…","""(20.0, 30.0]"""
"""NA""",112103,"""Instituto Naci…","""Pertenecientes…",null,12,1,null,null,null,56,null,null,null,13101,"""Santiago""","""13""","""Metropolitana …",1,"""A""",null,null,2001,2001-03-12,111,41,2,"""J189""","""NEUMONIA, NO E…",null,null,2,null,null,null,null,null,"""Región Metropo…","""Santiago, Regi…","""(50.0, 60.0]"""
"""NA""",112103,"""Instituto Naci…","""Pertenecientes…",null,12,1,null,null,null,18,null,null,null,13101,"""Santiago""","""13""","""Metropolitana …",99,null,null,null,2001,2001-11-29,121,49,2,"""D381""","""TUMOR DE COMPO…",null,null,1,null,null,null,null,null,"""Región Metropo…","""Santiago, Regi…","""(10.0, 20.0]"""
